In [140]:
import ipywidgets as widgets
from ipywidgets import HBox, VBox
from IPython.display import display, clear_output

In [141]:
from sqlalchemy import create_engine, Column, Integer, String, Date, Time, ForeignKey
from sqlalchemy.orm import sessionmaker, declarative_base, relationship
from datetime import time as dtime

In [142]:
Base = declarative_base()
engine = create_engine('sqlite:///lifeplannerdatabase.db')

In [143]:
class User(Base):
    __tablename__ = 'users'
    id = Column(Integer, primary_key=True, autoincrement=True)
    name = Column(String, nullable=False, unique =True)
    tasks = relationship('Task', back_populates='user', cascade='all, delete')
    def __repr__(self):
        return f'<User(id={self.id}, name={self.name})>'

In [144]:
class Task(Base):
    __tablename__ = 'tasks'
    id = Column(Integer, primary_key=True, autoincrement=True)
    user_id = Column(Integer, ForeignKey('users.id'), nullable=False)
    user = relationship('User', back_populates='tasks')
    date = Column(Date, nullable=False)
    time = Column(Time)
    title = Column(String, nullable=False)
    description = Column(String)
    def __repr__(self):
        return f'<Task(id={self.id}, user_id={self.user_id}, date={self.date}, title={self.title})>'

In [145]:
Base.metadata.create_all(engine)
Session = sessionmaker(bind=engine)
session = Session()

In [146]:
current_user = None
current_date = None

output_area = widgets.Output()


In [147]:
user_input = widgets.Text(placeholder = 'Full Name', description = "User:")
load_button = widgets.Button(description="Enter", button_style="success")
date = widgets.DatePicker(description = 'Pick a Date')
tasks = widgets.SelectMultiple(description = 'Tasks', options = [])
add_button = widgets.Button(description="Add Task", button_style="success")
edit_button = widgets.Button(description="Edit Task")
delete_button = widgets.Button(description="Remove Task", button_style="warning")
confirm_delete_button = widgets.Button(description="Are you sure you want the selected task(s) to be removed?", button_style="danger", disabled=True)

In [148]:
pre_structured_ui = widgets.VBox([
    user_input,
    date,
    load_button,
    output_area
])

In [149]:
structured_ui = widgets.VBox([
    user_input,
    widgets.HBox([date, load_button]),
    tasks,
    widgets.HBox([add_button, edit_button, delete_button]),
    output_area
    ])

In [150]:
def on_button_clicked(b):
    user_name = user_input.value
    chosen_date = date.value
    with output_area:
        clear_output()
        if not user_name or not chosen_date:
            print('Please enter your full name and select the date before you click Enter')
            return

        user = session.query(User).filter_by(name=user_name).first()
        if user is None:
            user = User(name=user_name)
            session.add(user)
            session.commit()
        daily_tasks = session.query(Task).filter_by(user_id=user.id, date=chosen_date).all()
        tasks.options = [task.title for task in daily_tasks]
        container.children = [structured_ui]

load_button.on_click(on_button_clicked)

In [151]:
container = widgets.VBox([pre_structured_ui])
display(container)